<a href="https://colab.research.google.com/github/TitapornEve/twitter_Crawler/blob/main/%E0%B8%95%E0%B8%B1%E0%B8%94%E0%B8%84%E0%B8%B3%E0%B8%AB%E0%B8%A2%E0%B8%B2%E0%B8%9A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pythainlp
!pip install emoji

     |████████████████████████████████| 10.6MB 4.4MB/s 
     |████████████████████████████████| 747kB 22.6MB/s 
     |████████████████████████████████| 133kB 5.5MB/s 


In [3]:
from nltk import NaiveBayesClassifier as nbc
from sklearn.feature_extraction.text import CountVectorizer

from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords

import codecs

from itertools import chain

import pandas as pd
import numpy as np

import emoji
import re

from wordcloud import WordCloud
import matplotlib.pyplot as plt 

In [11]:
tweet=pd.read_excel("twitterCrawler55.xlsx")


In [12]:
tweet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    19 non-null     object
dtypes: object(1)
memory usage: 280.0+ bytes


In [13]:
tweet.describe()

,text
count,19
unique,10
top,พักขายกล้อง ขอด่ามหาลัย รู้ว่าอยู่ในพื้นที่เสี...
freq,8


In [14]:
tweet.head()

,text
0,อยากโดนเย็ด #มอปลาย\n#ชายแท้อยากลอง #ชายแท้ #อ...
1,ขอเชิญพี่น้องชาวคลองหกร่วมลงทะเบียนแบบฟอร์มเรี...
2,ขอเชิญพี่น้องชาวคลองหกร่วมลงทะเบียนแบบฟอร์มเรี...
3,หาเพื่อนเรียนคณะสื่อสารมวลชนค่า โฆษณายิ่งดีเลย...
4,กินข้าวไหนดีวะ ที่ไหนก็อันตราย😂😂😂 #คลองหก #ราช...


In [15]:
tweet.drop_duplicates("text")

,text
0,อยากโดนเย็ด #มอปลาย\n#ชายแท้อยากลอง #ชายแท้ #อ...
1,ขอเชิญพี่น้องชาวคลองหกร่วมลงทะเบียนแบบฟอร์มเรี...
3,หาเพื่อนเรียนคณะสื่อสารมวลชนค่า โฆษณายิ่งดีเลย...
4,กินข้าวไหนดีวะ ที่ไหนก็อันตราย😂😂😂 #คลองหก #ราช...
5,พักขายกล้อง ขอด่ามหาลัย รู้ว่าอยู่ในพื้นที่เสี...
7,ขอฉอดหน่อยเถอะค่ะ เรียนออนไลน์เนี่ย เรียนมาได้...
11,คือต้องมีเด็กในคณะติดก่อนหรอ คืองงอ่ะ นี่สอบถา...
16,อยากอมควยชายแท้ #มอปลาย\n#ชายแท้อยากลอง #ชาย...
17,อยากอมควยชายแท้ #มอปลาย\n#ชายแท้อยากลอง #ชาย...
18,หาเพื่อนคณะบริหาร สาขาโลจิสค้าบบบ เราไปเรียนคน...


In [16]:
tweet.text

0     อยากโดนเย็ด #มอปลาย\n#ชายแท้อยากลอง #ชายแท้ #อ...
1     ขอเชิญพี่น้องชาวคลองหกร่วมลงทะเบียนแบบฟอร์มเรี...
2     ขอเชิญพี่น้องชาวคลองหกร่วมลงทะเบียนแบบฟอร์มเรี...
3     หาเพื่อนเรียนคณะสื่อสารมวลชนค่า โฆษณายิ่งดีเลย...
4     กินข้าวไหนดีวะ ที่ไหนก็อันตราย😂😂😂 #คลองหก #ราช...
5     พักขายกล้อง ขอด่ามหาลัย รู้ว่าอยู่ในพื้นที่เสี...
6     พักขายกล้อง ขอด่ามหาลัย รู้ว่าอยู่ในพื้นที่เสี...
7     ขอฉอดหน่อยเถอะค่ะ เรียนออนไลน์เนี่ย เรียนมาได้...
8     พักขายกล้อง ขอด่ามหาลัย รู้ว่าอยู่ในพื้นที่เสี...
9     พักขายกล้อง ขอด่ามหาลัย รู้ว่าอยู่ในพื้นที่เสี...
10    พักขายกล้อง ขอด่ามหาลัย รู้ว่าอยู่ในพื้นที่เสี...
11    คือต้องมีเด็กในคณะติดก่อนหรอ คืองงอ่ะ นี่สอบถา...
12    คือต้องมีเด็กในคณะติดก่อนหรอ คืองงอ่ะ นี่สอบถา...
13    พักขายกล้อง ขอด่ามหาลัย รู้ว่าอยู่ในพื้นที่เสี...
14    พักขายกล้อง ขอด่ามหาลัย รู้ว่าอยู่ในพื้นที่เสี...
15    พักขายกล้อง ขอด่ามหาลัย รู้ว่าอยู่ในพื้นที่เสี...
16    อยากอมควยชายแท้  #มอปลาย\n#ชายแท้อยากลอง  #ชาย...
17    อยากอมควยชายแท้  #มอปลาย\n#ชายแท้อยากลอง  

In [17]:
with codecs.open('pos.txt', 'r', "utf-8") as f:
    lines = f.readlines()
listpos=[e.strip() for e in lines]
del lines
f.close() # ปิดไฟล์

In [19]:
with codecs.open('neg.txt', 'r', "utf-8") as f:
    lines = f.readlines()
listneg=[e.strip() for e in lines]
f.close() # ปิดไฟล์

In [20]:
pos1=['ไม่หยาบ']*len(listpos)
neg1=['หยาบ']*len(listneg)

data = list(zip(listpos,pos1)) + list(zip(listneg,neg1))

vocabulary = set(chain(*[word_tokenize(i[0].lower()) for i in data]))


feature_set = [({i:(i in word_tokenize(sentence.lower())) for i in vocabulary},tag) for sentence, tag in data]

classifier = nbc.train(feature_set)

#while True:
for x in tweet.text:
  test_sentence= x
  featurized_test_sentence={i:(i in word_tokenize(test_sentence.lower()))for i in vocabulary}
  print("test_sent:",test_sentence)
  print("ตัดคำ:",word_tokenize(test_sentence))
  print("tag:",classifier.classify(featurized_test_sentence))
  print("\n")

test_sent: อยากโดนเย็ด #มอปลาย
#ชายแท้อยากลอง #ชายแท้ #อมควย #อมควยชายแท้  #คลอง6 #ราชมงคลธัญบุรี #คลองหก #ความลับ  #ชายแท้รับงานกทม #ชายแท้เงี่ยนควย #ชายแท้รับงานเกย์ #ชายแท้ชอบสาวสอง #ชักว่าว #มัธยมขี้เงี่ยน #มัธยมนัดเย็ด  #ธัญบุรี #อยากลอง #ว่าว #ราชมงคล #อยากโดนเย็ด
ตัดคำ: ['อยาก', 'โดน', 'เย็ด', ' ', '#มอ', 'ปลาย', '\n', '#', 'ชาย', 'แท้', 'อยาก', 'ลอง', ' ', '#', 'ชาย', 'แท้', ' ', '#อม', 'ควย', ' ', '#อม', 'ควย', 'ชาย', 'แท้', '  ', '#', 'คลอง', '6', ' ', '#', 'ราชมงคล', 'ธัญบุรี', ' ', '#', 'คลอง', 'หก', ' ', '#', 'ความลับ', '  ', '#', 'ชาย', 'แท้', 'รับงาน', 'กทม', ' ', '#', 'ชาย', 'แท้', 'เงี่ยน', 'ควย', ' ', '#', 'ชาย', 'แท้', 'รับงาน', 'เกย์', ' ', '#', 'ชาย', 'แท้', 'ชอบ', 'สาว', 'สอง', ' ', '#', 'ชักว่าว', ' ', '#', 'มัธยม', 'ขี้', 'เงี่ยน', ' ', '#', 'มัธยม', 'นัด', 'เย็ด', '  ', '#', 'ธัญบุรี', ' ', '#', 'อยาก', 'ลอง', ' ', '#', 'ว่าว', ' ', '#', 'ราชมงคล', ' ', '#', 'อยาก', 'โดน', 'เย็ด']
tag: หยาบ


test_sent: ขอเชิญพี่น้องชาวคลองหกร่วมลงทะเบียนแบบฟอร์มเรียกคืนค่าเทอม